In [2]:
import pandas as pd
import numpy as np
import requests
import os
from dotenv import load_dotenv
import re
import time

In [3]:
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [4]:
busan_restrau_dir = "/mnt/t7/dnn/llm_practicing/05_RAG_practicing/datas/busan_restrau_20to24.csv"
busan_restrau = pd.read_csv(busan_restrau_dir)

In [5]:
busan_restrau.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 626 entries, 0 to 625
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SYS_NO           626 non-null    int64  
 1   INDUTY_NM        626 non-null    object 
 2   BZCND_NM         626 non-null    object 
 3   RSTRNT_NM        626 non-null    object 
 4   LOCPLC_ALL_ADDR  626 non-null    object 
 5   LOCPLC_LNM_ADDR  626 non-null    object 
 6   MENU_NM          626 non-null    object 
 7   TEL_NO           626 non-null    object 
 8   APPN_DE          626 non-null    object 
 9   RSNMT_DE         626 non-null    object 
 10  GUGUN_NM         626 non-null    object 
 11  DATA_BASE_DE     626 non-null    object 
 12  GID_LA           626 non-null    float64
 13  GID_LO           626 non-null    float64
dtypes: float64(2), int64(1), object(11)
memory usage: 68.6+ KB


In [6]:
busan_restrau.head()

,SYS_NO,INDUTY_NM,BZCND_NM,RSTRNT_NM,LOCPLC_ALL_ADDR,LOCPLC_LNM_ADDR,MENU_NM,TEL_NO,APPN_DE,RSNMT_DE,GUGUN_NM,DATA_BASE_DE,GID_LA,GID_LO
0,0,일반음식점,식육(숯불구이),행복한빅마마,부산광역시 연제구 월드컵대로 151 (연산동),부산광역시 연제구 연산동 1241번지 9호,돼지갈비,051-852-0218,2014-10-30 12:00,2018-10-26 12:00,연제구,2019-05-07 12:00,35.187,129.080
1,1,일반음식점,한식,해운대 신 대구탕,"부산광역시 연제구 월드컵대로 146 (연산동,1층)",부산광역시 연제구 연산동 1127번지 5호 1층,대구탕,051-867-7611,2011-11-09 12:00,2018-10-26 12:00,연제구,2019-05-07 12:00,35.187,129.080
2,2,일반음식점,뷔페식,해암뷔페,"부산광역시 연제구 중앙대로 1099 (연산동,,13)","부산광역시 연제구 연산동 1242번지 8호 ,13",뷔페,051-867-7600,2010-06-30 12:00,2018-10-26 12:00,연제구,2019-05-07 12:00,35.186,129.081
3,3,일반음식점,한식,할매보쌈,부산광역시 연제구 월드컵대로145번길 11 (연산동),부산광역시 연제구 연산동 1242번지 6호,보쌈,051-853-8005,2010-06-30 12:00,2018-10-26 12:00,연제구,2019-05-07 12:00,35.186,129.080
4,4,일반음식점,중국식,팔선각,부산광역시 연제구 법원로32번길 10 (거제동),부산광역시 연제구 거제동 1490번지 2호,중식,051-507-9555,2010-06-30 12:00,2018-10-26 12:00,연제구,2019-05-07 12:00,35.191,129.075


In [29]:
tel_no = busan_restrau.loc[busan_restrau['RSTRNT_NM'] == '명동닭한마리', 'TEL_NO'].values
type(tel_no)

numpy.ndarray

In [7]:
busan_restrau_format = busan_restrau[['BZCND_NM', 'RSTRNT_NM', 'LOCPLC_ALL_ADDR', 'MENU_NM', 'TEL_NO', 'GUGUN_NM', 'GID_LA', 'GID_LO']].copy()
busan_restrau_format.rename(columns={'BZCND_NM':'MENU_TYPE', 'RSTRNT_NM':'RESTAURANT_NAME_KOREAN', 'LOCPLC_ALL_ADDR': 'ADDRESS_KOREAN', 'MENU_NM':'MENU_NAME', 'TEL_NO':'NATIONAL_PHONE_NUMBER', 'GUGUN_NM':'DISTRICT_NAME', 'GID_LA':'LATITUDE', 'GID_LO':'LONGITUDE'},inplace = True)
busan_restrau_format.head()

,MENU_TYPE,RESTAURANT_NAME_KOREAN,ADDRESS_KOREAN,MENU_NAME,NATIONAL_PHONE_NUMBER,DISTRICT_NAME,LATITUDE,LONGITUDE
0,식육(숯불구이),행복한빅마마,부산광역시 연제구 월드컵대로 151 (연산동),돼지갈비,051-852-0218,연제구,35.187,129.080
1,한식,해운대 신 대구탕,"부산광역시 연제구 월드컵대로 146 (연산동,1층)",대구탕,051-867-7611,연제구,35.187,129.080
2,뷔페식,해암뷔페,"부산광역시 연제구 중앙대로 1099 (연산동,,13)",뷔페,051-867-7600,연제구,35.186,129.081
3,한식,할매보쌈,부산광역시 연제구 월드컵대로145번길 11 (연산동),보쌈,051-853-8005,연제구,35.186,129.080
4,중국식,팔선각,부산광역시 연제구 법원로32번길 10 (거제동),중식,051-507-9555,연제구,35.191,129.075


In [32]:
def clean_ktext(text):
    cleaned_text = re.sub(r"[^\w가-힣]", "", text)
    return cleaned_text


def get_place_id(name, location):
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "key": GOOGLE_API_KEY,
        "location": location,
        "radius": 200,  # 검색 반경 (미터 단위)
        "keyword": name,  # 레스토랑 이름
        "type": "restaurant"
    }
    
    # Google Places API 요청
    response = requests.get(url, params=params)
    print(response.json())
    if response.status_code == 200:
        data = response.json()
        
        # 검색 결과가 있을 경우 이름 일치 여부 확인
        if "results" in data and len(data["results"]) > 0:
            for result in data["results"]:
                # 결과의 이름과 입력한 이름이 정확히 일치하는지 확인
                if clean_ktext(result["name"]) == clean_ktext(name):
                    return result["place_id"]
        
        return None
    else:
        return None
    
def get_place_id_by_phone(phone_number):
    url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
    params = {
        "key": GOOGLE_API_KEY,
        "input": phone_number,
        "inputtype": "phonenumber",
        "fields": "place_id"
    }
    
    # Google Places API 요청
    response = requests.get(url, params=params)
    print(response.json())
    if response.status_code == 200:
        data = response.json()
        
        # 검색 결과가 있을 경우 place_id 반환
        if "candidates" in data and len(data["candidates"]) > 0:
            return data["candidates"][0]["place_id"]
        else:
            return "검색 결과 없음"
    else:
        return f"에러: {response.status_code}"
    
def get_place_details_new(place_id):
    details_url = f"https://places.googleapis.com/v1/places/{place_id}"
    details_params = {
        'fields': 'displayName,types,formattedAddress,priceLevel,rating,userRatingCount,regularOpeningHours,accessibilityOptions,goodForChildren,goodForGroups,menuForChildren,parkingOptions,paymentOptions,outdoorSeating,reservable,restroom,servesBeer,servesBreakfast,servesBrunch,servesDinner,servesLunch,servesVegetarianFood,takeout,reviews',
        'key': GOOGLE_API_KEY
    }
    response = requests.get(details_url, params=details_params)
    return response.json()




def format_review(response):
    if not response: return None
    total_review = ''
    for review in response:
        total_review += review.get('text').get('text')  + '\n' if review.get('text') else ''
    return total_review

def format_multiple_options(response):
    if not response: return None
    true_options = [k for k, v in response.items() if v]
    return ', '.join(true_options)

# def format_opening_hours(response):
#     if response.get('regularOpeningHours'): return None
#     return '\n'.join([re.sub(r'[^\x00-\x7F–]', '', text) for text in response.get('regularOpeningHours').get('weekdayDescriptions')])

def format_opening_hours(response):
    if not response.get('regularOpeningHours'): 
        print("None!!!")
        return None
    return '\n'.join([re.sub(r'[^\x00-\x7F–]', '', text) for text in response.get('regularOpeningHours').get('weekdayDescriptions')])

In [37]:
# 토곡식육식당
# 35.183,129.106
# 35.1827394,129.1039467

# 행복한빅마마
# 35.187,129.08

# 명동닭한마리
# 35.134387,129.086616

# 해운대 신 대구탕
# 35.187,129.080

# restaurant_name = "토곡식육식당"
# location = "35.1827394,35.1827394" 

restaurant_name = "해운대 신 대구탕"
location = "35.187,129.080" 

# Step 1: Get Place ID for the restaurant
place_id = get_place_id(restaurant_name, location)

if place_id:
    details = get_place_details_new(place_id)
    
    for key, value in details.items():
        if key != 'kreviews':
            print(f"{key}; {value}")
    
    # Print the reviews (optional)
    print("\nReviews:")
    print(details.get("reviews"))
    formatted_review = format_review(details.get("reviews"))
    print(formatted_review)

else:
    print("Restaurant not found.")

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 35.186776, 'lng': 129.0804187}, 'viewport': {'northeast': {'lat': 35.18812582989272, 'lng': 129.0817685298927}, 'southwest': {'lat': 35.18542617010728, 'lng': 129.0790688701072}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png', 'icon_background_color': '#FF9E67', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet', 'name': '해운대신대구탕', 'opening_hours': {'open_now': False}, 'photos': [{'height': 2268, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/105792666354829266924">young mi Kwon</a>'], 'photo_reference': 'AdCG2DP6XZxflkeLnCRA97Imu6lrp6nVdoT2BK6aMaPnEpYim_deAnpnLAHNCpfhoxmJpRWyCHAAbmo_ZdLohrNm1UNWLLxZz4EOBhVjM0tNahKplk7Cbt1BFVXov2Tkuw4xVJtrby-xTIYZY308KxpEgi39JjYbL-xlQqbLsCX8oWbbyOVk', 'width': 4032}], 'place_id': 'ChIJi2YUIVuTaDURxOFKvjydesE', 'plus_code': {'compound_code': 

In [40]:
tel_num = "0516362834"

get_place_id_by_phone(tel_num)

{'candidates': [], 'status': 'INVALID_REQUEST'}


'검색 결과 없음'

In [33]:
Nan_id_rows_df = pd.DataFrame(columns=busan_restrau_format.columns)

for i, row in busan_restrau_format.iterrows():
    time.sleep(0.2)
    # restaurant_name = row["RESTAURANT_NAME_KOREAN"]
    # location = f"{row['LATITUDE']},{row['LONGITUDE']}" 
    # place_id = get_place_id(restaurant_name, location)
    tel_num = row["NATIONAL_PHONE_NUMBER"]
    get_place_id_by_phone(tel_num)
    print(tel_num)


    if i > 2:
        break

    # if place_id:
    #     details = get_place_details_new(place_id)
    #     print(details)

    #     busan_restrau_format.loc[i, 'RESTAURANT_NAME_ENGLISH'] = details.get('displayName').get('text') if details.get('displayName') and details.get('displayName').get('languageCode')=='en' else None
    #     busan_restrau_format.loc[i, 'ADDRESS_ENGLISH'] = details.get('formattedAddress')
    #     busan_restrau_format.loc[i, 'OPENING_HOURS'] = format_opening_hours(details)
    #     busan_restrau_format.loc[i, 'RATING'] = details.get('rating')
    #     busan_restrau_format.loc[i, 'RATING_COUNT'] = details.get('userRatingCount')
    #     busan_restrau_format.loc[i, 'PRICE_LEVEL'] = details.get('priceLevel')
    #     busan_restrau_format.loc[i, 'TAKEOUT_YN'] = details.get('takeout')
    #     busan_restrau_format.loc[i, 'RESERVABLE'] = details.get('reservable')
    #     busan_restrau_format.loc[i, 'BREAKFAST_YN'] = details.get('servesBreakfast')
    #     busan_restrau_format.loc[i, 'LUNCH_YN'] = details.get('servesLunch')
    #     busan_restrau_format.loc[i, 'DINNER_YN'] = details.get('servesDinner')
    #     busan_restrau_format.loc[i, 'BEER_YN'] = details.get('servesBeer')
    #     busan_restrau_format.loc[i, 'OUTDOOR_SEAT_YN'] = details.get('outdoorSeating')
    #     busan_restrau_format.loc[i, 'MENU_FOR_CHILDREN_YN'] = details.get('menuForChildren')
    #     busan_restrau_format.loc[i, 'RESTROOM_YN'] = details.get('restroom')
    #     busan_restrau_format.loc[i, 'PARKING_LOT_YN'] = format_multiple_options(details.get('parkingOptions'))
    #     busan_restrau_format.loc[i, 'PAYMENT_OPTIONS'] = format_multiple_options(details.get('paymentOptions'))
    #     busan_restrau_format.loc[i, 'REVIEW'] = format_review(details.get("reviews"))
        
    # else:
    #     print("None!!!!!!!!!!!!!")
    #     Nan_id_rows_df = pd.concat([Nan_id_rows_df, row.to_frame().T], ignore_index=True)
    #     continue

{'candidates': [], 'status': 'INVALID_REQUEST'}
051-852-0218
{'candidates': [], 'status': 'INVALID_REQUEST'}
051-867-7611
{'candidates': [], 'status': 'INVALID_REQUEST'}
051-867-7600
{'candidates': [], 'status': 'INVALID_REQUEST'}
051-853-8005


In [ ]:
# busan_restrau_format.info()
# busan_restrau_format.to_csv("./datas/busan_restrau_20to24_witch_eng_data.csv", index=False, encoding="utf-8")
# Nan_id_rows_df.to_csv("./datas/busan_restrau_20to24_none_id_data.csv", index=False, encoding="utf-8")